In [1]:
import tensorflow as tf
import numpy as np
import os
import time
tf.enable_eager_execution()

In [2]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file="斗罗大陆.txt"
text = open(path_to_file).read()
print(text[:1000])
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

斗罗大陆
作者：唐家三少
本书来自【无名txt小说下载网-www.wmtxt.com】站内精心编辑制作 提供免费下载多谢你的支持！本站所有书籍均来自网络收集，站内精心编辑制作 提供免费下载。版权属作者或出版社所有。  精彩全本TXT小说下载请登www.wmtxt.com
第一集 斗罗世界  
引子 穿越的唐家三少
巴蜀，历来有天府之国的美誉，其中，最有名的门派莫过于唐门。
唐门所在是一个神秘的地方，许多人只知道，那是一个半山腰，而唐门所在这座山的山顶有一个令人胆颤心惊的名字，——鬼见愁。
从鬼见愁悬崖上扔出一块石头，要足足数上十九下才会听到石落山底的回声，可见其高，也正是因为这十九秒，尚超过十八层地狱一筹，故而得名。
一名身穿灰衣的青年正站在鬼见愁顶峰，凛冽的山风不能令他的身体有丝毫移动，从他胸口处那斗大的唐字就可以认出，他来自唐门，灰衣代表的，是唐门外门弟子。
他今年二十九岁，因出生不久就进入唐门，在外门弟子的辈分中排名第三，因此外门弟子称他一声三少。当然，到了内门弟子口中，就变成了唐三。
唐门从建立时开始就分为内外两门，外门都是外姓或被授予唐姓的弟子，而内门，则是唐门直系所属，家族传承。
此时，唐三脸上的表情很丰富，时而笑，时而哭，但无论如何，都无法掩盖他的那发自内心的兴奋。
二十九年了，自从二十九年前他被外门长老唐蓝太爷在襁褓时就捡回唐门时开始，唐门就是他的家，而唐门的暗器就是他的一切。
突然，唐三脸色骤然一变，但很快又释然了，有些苦涩的自言自语道：“该来的终究还是来了。”
十七道身影，十七道白色的身影，宛如星丸跳跃一般从山腰处朝山顶方向而来，这十七道身影的主人，年纪最小的也超过了五旬，一个个神色凝重，他们身穿的白袍代表的是内门，而胸前那金色的唐字则是唐门长老的象征。
唐门内门长老堂包括掌门唐大先生在内，一共有十七位长老，此时登山的，也正是十七位。就算是武林大会也不可能惊动唐门全部长老同时出动，要知道，这唐门长老之中，年纪最大的已经超过了两个甲子。
这些唐门长老的修为，无一不是已臻化境，只是转眼的工夫，他们就已经来到了山顶。
外门弟子见到内门长老，只有跪倒迎接的份，但此时，唐三却没有动，他只是静静的看着这些脸色凝重的长老来到自己面前，挡住了所有的去路，而在他背后，是鬼见愁。
放下三朵佛怒唐莲，唐三投下最后那恋恋不舍的一眼，嘴角处流露出一丝欣慰的微笑，他毕竟成功

In [3]:
seq_length = 100
chunks = tf.data.Dataset.from_tensor_slices(text_as_int).batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
dataset = chunks.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [4]:
class Model(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, units):
    super(Model, self).__init__()
    self.units = units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                     return_sequences=True,
                                     recurrent_activation='sigmoid',
                                     recurrent_initializer='glorot_uniform',
                                     stateful=True)
    self.fc = tf.keras.layers.Dense(vocab_size)
  def call(self, x):
    embedding = self.embedding(x)
    output = self.gru(embedding)
    prediction = self.fc(output)
    return prediction
vocab_size = len(vocab)
embedding_dim = 256
units = 1024
model = Model(vocab_size, embedding_dim, units)
optimizer = tf.train.AdamOptimizer()
def loss_function(real, preds):
    return tf.losses.sparse_softmax_cross_entropy(labels=real, logits=preds)
model.build(tf.TensorShape([BATCH_SIZE, seq_length]))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1009152   
_________________________________________________________________
gru (GRU)                    multiple                  3935232   
_________________________________________________________________
dense (Dense)                multiple                  4040550   
Total params: 8,984,934
Trainable params: 8,984,934
Non-trainable params: 0
_________________________________________________________________


In [12]:
checkpoint_dir = './cnm_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
model = Model(vocab_size, embedding_dim, units)
model.load_weights(checkpoint_prefix)
model.build(tf.TensorShape([BATCH_SIZE, seq_length]))
EPOCHS = 50
for epoch in range(EPOCHS):
    start = time.time()
    hidden = model.reset_states()
    for (batch, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              predictions = model(inp)
              loss = loss_function(target, predictions)
          grads = tape.gradient(loss, model.variables)
          optimizer.apply_gradients(zip(grads, model.variables))
          if batch % 100 == 0:
              print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch+1,batch,loss))
    if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix)
    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
model.save_weights(checkpoint_prefix)

Epoch 1 Batch 0 Loss 1.8595
Epoch 1 Batch 100 Loss 3.0639
Epoch 1 Batch 200 Loss 2.9052
Epoch 1 Batch 300 Loss 2.8013
Epoch 1 Batch 400 Loss 2.6918
Epoch 1 Loss 2.8856
Time taken for 1 epoch 279.09106969833374 sec

Epoch 2 Batch 0 Loss 2.1986
Epoch 2 Batch 100 Loss 2.5323
Epoch 2 Batch 200 Loss 2.5210
Epoch 2 Batch 300 Loss 2.4448
Epoch 2 Batch 400 Loss 2.3975
Epoch 2 Loss 2.5993
Time taken for 1 epoch 278.71822237968445 sec

Epoch 3 Batch 0 Loss 1.9575
Epoch 3 Batch 100 Loss 2.3244
Epoch 3 Batch 200 Loss 2.3276
Epoch 3 Batch 300 Loss 2.2708
Epoch 3 Batch 400 Loss 2.2509
Epoch 3 Loss 2.4248
Time taken for 1 epoch 278.7239639759064 sec

Epoch 4 Batch 0 Loss 1.8292
Epoch 4 Batch 100 Loss 2.2210
Epoch 4 Batch 200 Loss 2.2110
Epoch 4 Batch 300 Loss 2.1791
Epoch 4 Batch 400 Loss 2.1606
Epoch 4 Loss 2.3225
Time taken for 1 epoch 278.7929754257202 sec

Epoch 5 Batch 0 Loss 1.7828
Epoch 5 Batch 100 Loss 2.1333
Epoch 5 Batch 200 Loss 2.1469
Epoch 5 Batch 300 Loss 2.0983
Epoch 5 Batch 400 Loss 2

Epoch 38 Batch 400 Loss 1.6317
Epoch 38 Loss 1.7529
Time taken for 1 epoch 277.2389483451843 sec

Epoch 39 Batch 0 Loss 1.4815
Epoch 39 Batch 100 Loss 1.6512
Epoch 39 Batch 200 Loss 1.6501
Epoch 39 Batch 300 Loss 1.6182
Epoch 39 Batch 400 Loss 1.6125
Epoch 39 Loss 1.7463
Time taken for 1 epoch 277.439297914505 sec

Epoch 40 Batch 0 Loss 1.4707
Epoch 40 Batch 100 Loss 1.6529
Epoch 40 Batch 200 Loss 1.6342
Epoch 40 Batch 300 Loss 1.6309
Epoch 40 Batch 400 Loss 1.6034
Epoch 40 Loss 1.7346
Time taken for 1 epoch 277.63431692123413 sec

Epoch 41 Batch 0 Loss 1.4707
Epoch 41 Batch 100 Loss 1.6345
Epoch 41 Batch 200 Loss 1.6251
Epoch 41 Batch 300 Loss 1.6307
Epoch 41 Batch 400 Loss 1.6047
Epoch 41 Loss 1.7267
Time taken for 1 epoch 277.1708221435547 sec

Epoch 42 Batch 0 Loss 1.4623
Epoch 42 Batch 100 Loss 1.6271
Epoch 42 Batch 200 Loss 1.6181
Epoch 42 Batch 300 Loss 1.6349
Epoch 42 Batch 400 Loss 1.5950
Epoch 42 Loss 1.7220
Time taken for 1 epoch 277.0613114833832 sec

Epoch 43 Batch 0 Loss 

In [13]:
model = Model(vocab_size, embedding_dim, units)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
num_generate = 1000
start_string = '为'
input_eval = [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)
text_generated = []
temperature = 1.0
model.reset_states()
for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])
print (start_string + ''.join(text_generated))

为金色，单是这种都是十分熟悉的团长。一旦被他顶得杀神领域地地方，魔魂大白鲨都吃下了亢奋粉红肠，吞噬后打动了。而且现在只能从唐三释飞到六十级层层甲上破碎。当时，他心中却蒙着了一层阴清，一动身形的踏鬼影迷踪步而去。如果在前一世的他再将其阻挡他也发挥不出任何作用。单是唐三手中海神三叉戟上释放的血焰对唐三来说还是极为少见的。但他可以肯定，有不少魂师实力的。
一边说着，大海中被金色渲染的唐三眼底深不过低。看到它。”海德尔前方。喊出了一个询问，从上面凝聚着一层紫波瞬间爆发出先前那一击。瞬间破开泥土。将两株大树随之破碎而入。如同封号斗罗地实力有多么恐怖，就要将他完全笼罩在内。到整个身体脱层皮肤。
看到魂师胸前，那张看着奥斯卡道：“小三，你的第四考是什么？我等大量的消耗
别地意义是什么？”
奥斯卡如此声音似的，白沉香却久了不着两人，被唐三拦在晕船，海神三叉戟从最前面的左眼中吐出，只剩下由衷的巨大地草。这种方法也很容易感觉到她那强悍的能量波动。
暗魔邪神虎眼中已是焰光芒反而迎向了血脉，封死了海龙斗罗的时机。
杀戮之王的厉电爆发。属于唐三的攻击本体不只是因为他们的攻击力而惊慌。可实际上，他的龟甲盾也能释放到十万年级别。
这一次她想要释放比诸葛神弩的威力，首先就看不到她怎么办？怎么也说不清。但现在自己却流露着希望能够真正的将自己地所能改变自己的经脉，至少可能让小舞留在那里后。
玄天功内力涌入对方群体的行动，五万年级别地魂兽，哪怕是当年魂师或者大斗魂场外观的大多数都是以铁甲为为传。因为他们眼中的景象已经因此而沉。下错，当地唐三也不所能留手。但是，唐三可是准备第二次与你决定。他刚刚到武魂帝国的情况下。这次我们攻击是迅雷直接攻击降临，这样的强者会极少威胁。你们听到海星七圣柱的剑斗罗，到礼数正上方，请回家三平房。
仅仅是，这次并不多厅，众人再也忍不住笑出来了。三个时辰过去了。缓缓抬头，看向唐三，“如果我不能活着走过去，那么，我就告诉你。我是爱嘴上拼，哪怕是遇到危险的时候也难以无与了死。恶狠的对手也是空强的情况。在我心中。实早一定智的与唐三等人。在下一刻选定要说。谁能达到最终威力的。将自己最后地魂师组合以及时昂的消失，整个弩机仿佛都在这股能量压制下的情况被荡起。尽管大部分魂师都突破了九级。就在这突如象来看那里将群起来。
独孤博道：“现在我在一起的时候，已经成就了海神，如果回答不会手，那如此虚空，就